In [ ]:
import pandas as pd
import psycopg2
import json

In [ ]:
business_raw = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')
business_raw.index +=1

In [ ]:
business_raw.head()

In [ ]:
eval(business_raw["attributes"][2])


# Parse businesses ids

In [ ]:
business_ids = business_raw["business_id"].reset_index().set_index("business_id")["index"].to_dict()
del business_raw["business_id"]
business_table = business_raw.reset_index().rename(columns={'index':'id'})
business_table.head()

# Parse Locations

In [ ]:
locations = business_table[["id", "address", "city", "latitude", "longitude", "postal_code", "state"]]
del business_table["id"]
del business_table["address"]
del business_table["city"]
del business_table["latitude"]
del business_table["longitude"]
del business_table["postal_code"]
del business_table["state"]
locations = locations.rename(columns={'id':'business_id'})
locations.head()

In [ ]:
locations.head()

# Parse states

In [ ]:
states = locations["state"].unique()
states.sort()
states

In [ ]:
state_table = pd.Series(states, name="state")
state_table.index += 1
state_table = state_table.reset_index().rename(columns={"index":"id"})
state_table.head()

In [ ]:
state_dict = state_table.set_index("state")["id"].to_dict()
locations["state"] = locations["state"].apply(lambda s: state_dict[s])
locations = locations.rename(columns={'state':'state_id'})
locations.head()

# Parse cities

In [ ]:
city_table = locations[["city", "state_id"]].drop_duplicates()
city_table.index += 1
city_table = city_table.reset_index().rename(columns={"index":"id", "city":"name"})
city_table.head()

In [ ]:
city_dict = city_table.set_index(["name", "state_id"])["id"].to_dict()

In [ ]:
locations["city_id"] = locations[["city", "state_id"]].apply(lambda row: city_dict[(row[0], row[1])], axis=1, raw=True)
del locations["state_id"]
del locations["city"]
locations.head()

# Parse postal codes

In [ ]:
postal_codes_table = locations[["city_id", "postal_code"]].drop_duplicates()
postal_codes_table.index += 1
postal_codes_table = postal_codes_table.reset_index().rename(columns={"index":"id"})
postal_codes_table.head()

In [ ]:
postal_code_dict = postal_codes_table.set_index(["postal_code", "city_id"])["id"].to_dict()

In [ ]:
locations["postal_code_id"] = locations[["postal_code", "city_id"]].apply(lambda row: postal_code_dict[(row[0], row[1])], axis=1, raw=True)
del locations["city_id"]
del locations["postal_code"]
locations.head()

# Store in data base

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

# stores states
for name, id in state_dict.items():
    cur.execute("insert into state (id, name) values ({id}, '{name}')".format(id=id, name=name))
con.commit()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")